In [ ]:
from pathlib import Path
from scipy import stats

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns

np.set_printoptions(suppress=True)

pd.options.display.max_columns = None
pd.options.display.max_rows = None
pd.options.display.max_colwidth = None

from IPython.display import display

In [ ]:
path1 = Path(r"A:\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")
path2 = Path(r"C:\Users\rober\Documents\Workspace\Python\Masterarbeit\Kaggle Home Credit Datensatz")

if path1.is_dir():
    DATASET_DIR = path1
else:
    DATASET_DIR = path2

In [123]:
app_train = pd.read_csv(DATASET_DIR / "application_train.csv")
bureau = pd.read_csv(DATASET_DIR / "bureau.csv")
bureau_balance = pd.read_csv(DATASET_DIR / "bureau_balance.csv")
credit_card_balance = pd.read_csv(DATASET_DIR / "credit_card_balance.csv")

In [ ]:
a = pd.unique(app_train["SK_ID_CURR"])

In [ ]:
b = pd.unique(bureau["SK_ID_CURR"])

In [ ]:
c = np.intersect1d(a,b)

In [ ]:
d = np.setdiff1d(a,b)

In [ ]:
len(a)

In [ ]:
len(c)+len(d)

In [ ]:
t = bureau[["SK_ID_CURR","SK_ID_BUREAU"]][0:10]
t.drop(t[t.SK_ID_CURR == 215354].index,inplace=True)
t

In [ ]:
milestones = [i for i in range(100+1)]
for index, ID in enumerate(d):
    bureau.drop(bureau[bureau["SK_ID_CURR"] == ID].index, inplace = True)

    if round(index/len(d)) >= min(milestones):
        print("{0}%".format(index/len(d)))
        milestones.remove(min(milestones))

0.0%


TypeError: 'numpy.ndarray' object is not callable

In [ ]:
bureau.head()

In [ ]:
for index, row in bureau.iterrows():
    id = row["SK_ID_CURR"]
    try:
        t = app_train[app_train["SK_ID_CURR"] == id].values[0][1]
        row["Target"] = t
    except IndexError as e:
        print(id)
        row["Target"] = np.nan
    
    

In [ ]:
df = app_train[app_train["SK_ID_CURR"] == 215354].values[0][1]

In [ ]:
bureau_balance.head()

In [ ]:
credit_card_balance.head()

In [ ]:
bureau[bureau["SK_ID_CURR"] == 215354]